In [1]:
# 필요 패키지 설치
#!pip install -q pymysql


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
# 필요 패키지
import pymysql # python에서 sql
import pandas as pd  # 분석, 시각화
import seaborn as sns # 시각화
import numpy as np # 분석
import matplotlib.pyplot as plt # 시각화
import warnings # 경고 무시
warnings.filterwarnings(action='ignore')
from sqlalchemy import create_engine

%matplotlib inline

In [41]:
data = pd.read_csv('data/final_deposit_all_for_test.csv', encoding = 'cp949')
#data = data.fillna('NA')
#data.to_csv('data/final_deposit_all_for_test.csv', encoding = 'EUC-KR', index = False)

In [42]:
conn = pymysql.connect(host = 'host', user = 'user_name', password = 'pw', db = 'goods', charset = 'utf8')

In [33]:
cur = conn.cursor()

In [34]:
host = 'host'
user = 'user_name'
password = 'pw'
database = 'goods'

def get_conn():
    '''
        데이터베이스 연결
        returns
            db 커넥션 객체
    '''
    return pymysql.connect(   host    = host,
                                    user    = user,
                                    password= password,
                                    database= database,
                                    # select의 결과 집합은 [ {}, {}, {}, ...] 형태로 나옴, 지정
                                    cursorclass=pymysql.cursors.DictCursor
                                    )

# DB와 연결 중단하는 모듈
def close_conn( conn ):
    '''
        데이터베이스 연결 종료
        parameters
            - conn : 커넥션 객체
    '''
    if conn:
        conn.close()
    pass

In [35]:
# 쿼리 수행 함수
def get_sql( sql : str ):
    '''
        DQL  계열의 select 쿼리 수행
        parameters
            - sql : select 문
        returns
            - 결과 집합, DataFrame으로 리턴
    '''
    # 쿼리를 요청할대마다 => 접속 => 쿼리 => 닫기 반복
    conn = None
    results = pd.DataFrame() # 비어있는 데이터프레임 자료구조
    try:
        conn    = get_conn()
        results = pd.read_sql(sql, conn)
    except Exception as e:
        pass
    finally:
        close_conn( conn )
    return results

# create view 등 DDL 계열의 명령어를 수행하는 모듈
def exec_sql( sql : str ):
    '''
        DDL  계열의 create view 쿼리 수행
        parameters
            - sql : create view ~
        returns
            - None, 성공여부
    '''
    conn = None
    try:
        conn    = get_conn()
        with conn:
            with conn.cursor() as cursor:
                cursor.execute( sql )
    except Exception as e:
        print( e )
        pass
    pass

In [36]:
# connect with DB
get_conn()

In [39]:
# deposit_all_fot_test DB
get_sql('select * from final_deposit_all_for_test')

,company,goods_name,method,basic_rate,after_tax_rate,max_rate,real_rate,rate_type,range_type,renew_date,...,finish,change_rate,events,prior_etc,age_limit,environment,marry,pet,working,trip
0,오에스비저축은행,인터넷정기예금,NA,4.50%,3.81%,4.50%,NA,복리,6개월,2023-10-31,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,오에스비저축은행,인터넷정기예금,NA,4.50%,3.81%,4.50%,NA,단리,6개월,2023-10-31,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,오에스비저축은행,정기예금,NA,4.50%,3.81%,4.50%,NA,복리,6개월,2023-10-31,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,오에스비저축은행,정기예금,NA,4.50%,3.81%,4.50%,NA,단리,6개월,2023-10-31,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,조은저축은행,정기예금(서울본점),NA,4.40%,3.72%,4.40%,NA,단리,6개월,2023-11-02,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,신한저축은행,s-정기예금,NA,1.80%,1.52%,1.80%,NA,복리,36개월,2023-10-31,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1935,신한저축은행,s-정기예금,NA,1.80%,1.52%,1.80%,NA,단리,36개월,2023-10-31,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1936,머스트삼일저축은행,자유적립예금,NA,1.60%,1.35%,1.60%,NA,단리,36개월,2021-09-08,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1937,스타저축은행,스타e_정기예금,NA,1.00%,0.85%,1.00%,NA,단리,36개월,2023-11-13,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [44]:
close_conn(conn)

In [45]:
conn